Trying to use more than 1 latent variable

In [ ]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [ ]:
from gpfa_imputation.imputation import *
from gpfa_imputation.data_preparation import *

import torch
import pandas as pd
import numpy as np
from pathlib import Path
from pyprojroot import here
import matplotlib.pyplot as plt

from gpfa_imputation.utils import cache_disk

In [ ]:
def reset_seed():
    torch.manual_seed(27);
    np.random.seed(27);

In [ ]:
torch.manual_seed(27);
np.random.seed(27);
cache = True
cache_path = here() / ".cache"

## Load Data

take the first 200 rows from the Hainich dataset

In [ ]:
hai_path = Path("FLX_DE-Hai_FLUXNET2015_FULLSET_HH_2000-2012_1-4.csv")
hai_raw = pd.read_csv(here("data") / hai_path, na_values=["-9999", "-9999.99"], parse_dates=[0, 1], nrows=200)

In [ ]:
meteo_vars = {
    "TA_F": "TA",
    "SW_IN_F": "SW_IN",
    "LW_IN_F": "LW_IN",
    "VPD_F": "VPD",
    #"PA": "PA"
}

units = {
    'TA': '°C',
    'SW_IN': 'W m-2',
    'LW_IN': 'W m-2',
    'VPD': 'hPa'
}

hai = (hai_raw
       .rename(columns=meteo_vars)
       .set_index("TIMESTAMP_END")
       .loc[:, meteo_vars.values()])
hai

,TA,SW_IN,LW_IN,VPD
TIMESTAMP_END,,,,
2000-01-01 00:30:00,-0.60,0.0,302.475,0.222
2000-01-01 01:00:00,-0.65,0.0,302.475,0.122
2000-01-01 01:30:00,-0.58,0.0,301.677,0.090
2000-01-01 02:00:00,-0.51,0.0,301.677,0.110
2000-01-01 02:30:00,-0.49,0.0,301.677,0.102
...,...,...,...,...
2000-01-05 02:00:00,4.74,0.0,330.202,1.191
2000-01-05 02:30:00,4.75,0.0,330.202,1.057
2000-01-05 03:00:00,4.76,0.0,330.202,0.935


###  Computer

In [ ]:
reset_seed()
data_r_gaps = GPFADataTest(hai[:150]).add_random_missing()
data_c_gap = GPFADataTest(hai[:150]).add_gap(30, meteo_vars.values())

In [ ]:
cache_file_gaps = cache_path / "hai_diff_latents.pickle"
@cache_disk(cache_file_gaps)
def compute_multiple_latent():
    hai_r_gaps = [data_r_gaps.to_imp(latent_dims=i, cuda=False).impute(ret_self = True) for i in range(1,4)]
    hai_c_gap = [data_c_gap.to_imp(latent_dims=i,cuda=False).impute(ret_self = True) for i in range(1,4)]
    return hai_r_gaps, hai_c_gap

In [ ]:
hai_r_gaps, hai_c_gaps = compute_multiple_latent()

  0%|          | 0/100 [00:00<?, ?it/s]

/home/simone/anaconda3/envs/data-science/lib/python3.10/site-packages/gpytorch/lazy/triangular_lazy_tensor.py:136: UserWarning: torch.triangular_solve is deprecated in favor of torch.linalg.solve_triangularand will be removed in a future PyTorch release.
torch.linalg.solve_triangular has its arguments reversed and does not return a copy of one of the inputs.
X = torch.triangular_solve(B, A).solution
should be replaced with
X = torch.linalg.solve_triangular(A, B). (Triggered internally at  ../aten/src/ATen/native/BatchLinearAlgebra.cpp:2183.)
  res = torch.triangular_solve(right_tensor, self.evaluate(), upper=self.upper).solution
/home/simone/anaconda3/envs/data-science/lib/python3.10/site-packages/torch/autograd/__init__.py:173: UserWarning: operator() profile_node %840 : int[] = prim::profile_ivalue(%838)
 does not have profile information (Triggered internally at  ../torch/csrc/jit/codegen/cuda/graph_fuser.cpp:104.)
  Variable._execution_engine.run_backward(  # Calls into the C++ eng

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

## Random Gaps

In [ ]:
hai_r_gaps[0].display_results()

In [ ]:
hai_r_gaps[1].display_results()

In [ ]:
hai_r_gaps[2].display_results()

## Continous gap

In [ ]:
hai_c_gaps[0].display_results()

In [ ]:
hai_c_gaps[1].display_results()

In [ ]:
hai_c_gaps[2].display_results()

/home/simone/Documents/uni/Thesis/GPFA_imputation/gpfa_imputation/imputation.py:142: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_var = df[df.variable == var & df.is_missing]
/home/simone/Documents/uni/Thesis/GPFA_imputation/gpfa_imputation/imputation.py:142: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_var = df[df.variable == var & df.is_missing]
/home/simone/Documents/uni/Thesis/GPFA_imputation/gpfa_imputation/imputation.py:142: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_var = df[df.variable == var & df.is_missing]
/home/simone/Documents/uni/Thesis/GPFA_imputation/gpfa_imputation/imputation.py:142: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_var = df[df.variable == var & df.is_missing]
